In [73]:
import astroplan as ap
from astropy.coordinates import SkyCoord
from astropy.table import QTable
from astropy.time import Time
import astropy.units as u
from astroquery.gaia import Gaia

In [2]:
# # read in the text format version of the WDS and save as an Astropy QTable
# error_lines = []
# with open("wdsweb_summ2.txt", "r", newline="\n") as f:
#     for _ in range(5):
#         f.readline()
#     rows = []
#     for line in f.readlines():
#         try:
#             line = line.strip()  # remove EOL character
#             row = {}
#             row["ID"] = line[0:10]
#             row["Discoverer"] = line[10:17]
#             components_text = line[17:22]
#             if not components_text.strip() == "":
#                 row["Components"] = components_text
#             row["Num Obs"] = int(line[33:37])
#             row["Separation"] = (float(line[46:51]) + float(line[51:57])) / 2 * u.arcsec
#             row["Position Angle"] = int(line[42:45])
#             row["Mag A"] = float(line[58:63])
#             magb_text = line[63:69]
#             if not magb_text.strip() == ".":
#                 row["Mag B"] = float(magb_text)
#                 row["Mag Avg"] = (row["Mag A"] + row["Mag B"]) / 2
#             else:
#                 row["Mag Avg"] = row["Mag A"]
#             spectral_text = line[70:79]
#             if not spectral_text.strip() == "":
#                 row["Spectral Type"] = spectral_text
#             notes_text = line[107:111]
#             if not notes_text.strip() == "":
#                 row["Notes"] = notes_text
#             row["RA"] = Angle(f"{line[112:114]}h{line[114:116]}m{line[116:121]}s").to(u.deg)
#             row["Dec"] = Angle(f"{line[121:124]}d{line[124:126]}m{line[126:130]}s").to(u.deg)
#             pmra_text = line[80:84]
#             row["PM RA"] = u.Quantity(0 if pmra_text.strip() == "" else int(pmra_text), unit=u.mas / u.yr)
#             pmdec_text = line[84:88]
#             row["PM Dec"] = u.Quantity(0 if pmdec_text.strip() == "" else int(pmdec_text), unit=u.mas / u.yr)
#             rows.append(row)
#         except:
#             error_lines.append(line)
#             continue
# print(f"{len(error_lines)} errors and {len(rows)} lines successfully processed")
# print("Error lines")
# for line in error_lines:
#     print(line)

# wds = QTable(
#     rows,
#     names=[
#         "ID",
#         "Discoverer",
#         "Components",
#         "Num Obs",
#         "Separation",
#         "Position Angle",
#         "Mag A",
#         "Mag B",
#         "Mag Avg",
#         "Spectral Type",
#         "Notes",
#         "RA",
#         "Dec",
#         "PM RA",
#         "PM Dec",
#     ],
# )
# wds["Separation"].info.format = ".2f"
# wds["Mag A"].info.format = ".2f"
# wds["Mag B"].info.format = ".2f"
# wds["Mag Avg"].info.format = ".2f"
# wds["RA"].info.format = "12.8f"
# wds["Dec"].info.format = "13.8f"
# wds.write("wdsweb_summ2.ecsv", overwrite=True)
# wds

In [3]:
wds = QTable.read("wdsweb_summ2.ecsv")
wds


ID,Discoverer,Components,Num Obs,Separation,Position Angle,Mag A,Mag B,Mag Avg,Spectral Type,Notes,RA,Dec,PM RA,PM Dec
,,,,arcsec,,,,,,,deg,deg,mas / yr,mas / yr
str10,str7,str5,int32,float64,int32,float64,float64,float64,str9,str4,float64,float64,float64,float64
00000+7530,A 1248,--,5,0.70,235,10.27,11.50,10.88,A7IV,--,0.02766667,75.48327778,34.0,5.0
00000+4004,ES 2543,AB,6,4.60,253,12.10,13.10,12.60,--,--,0.01525000,40.08872222,8.0,-4.0
00000+4004,ES 2543,AC,5,17.20,66,12.10,14.10,13.10,--,--,0.01525000,40.08872222,8.0,-4.0
00000+3852,BU 860,--,15,6.65,107,6.62,11.40,9.01,B9,--,0.00500000,38.85927778,3.0,-6.0
00000+0044,SKF1115,--,3,0.90,30,17.80,17.80,17.80,M1,--,359.99937500,0.72575000,-24.0,-20.0
00000-0530,OCC 622,--,0,0.10,-1,9.50,9.70,9.60,G5,--,0.00033333,-5.49436111,24.0,-12.0
00001+7727,LOC 122,--,6,8.15,249,14.10,16.60,15.35,--,T,0.03466667,77.44361111,86.0,37.0
00001+5400,ES 704,--,10,4.90,116,9.50,11.50,10.50,--,--,0.02850000,54.00005556,-30.0,3.0


# 2024-09-04 Updated procedure

As a result of the meeting, and emails with Jimmy, the following is the new procedure:

* Search the WDS for binaries, in RA 0h only, for the brightest match to the following criteria:
  * Separation under 0.5"
  * Magnitude brighter than 9
  * Magnitude difference less than 2
* Test if observable from APO during the A half of 2024-09-16.  If not, move on to the next brightest binary
* Search Gaia for a radius 110% of the separation of that binary, centered on the WDS coordinates
* If you don't get exactly 2 stars in that query, move on to the next brightest binary
* We now have Gaia's version of the binary.  Use the Gaia info for PA, separation, magnitudes, etc.
* From the midpoint of the Gaia binary, do a new Gaia search for point sources with the following criteria:
  * a 4' radius around the binary
  * within 1 mag of the brighter of the two components of the binary
* If there are no point sources found, move on to the next brightest binary
* For each of the possibly multiple point sources found, enter the binary & point source info in the targets table
* Move on to RA 1h and repeat the process
* Finish once the list of targets is a specified length, which might mean rotating through all the hours in RA multiple times

This avoids using coordinates from multiple catalogs simultaneously, and allows for the possibility of multiple point sources of varying separations for a given binary.

In [71]:
from astropy.coordinates import Angle
from astropy.table import QTable
import astropy.units as u
import numpy as np

# WDS binary candidate criteria
dimmest_binary = 9
brightest_binary = 3
widest_binary = 0.5 * u.arcsec
binary_mag_difference = 1
binary_criteria = (
    (wds["Mag Avg"] < dimmest_binary)
    & (wds["Mag Avg"] > brightest_binary)
    & (wds["Separation"] < widest_binary)
    & (abs(wds["Mag A"] - wds["Mag B"]) < binary_mag_difference)
)

# criteria to find matching gaia stars representing that binary
binary_match_cone = 5 * u.arcsec
binary_match_mag = 1

# point source criteria
point_match_mag = 1
point_match_sep_min = 1 * u.arcsec # in reality it is 0.45", but Jimmy wants to be conservative
point_match_sep_mid = 4 * u.arcmin
point_match_sep_max = 8 * u.arcmin

# list size & observation details
num_targets = 30
observer = ap.Observer.at_site("Apache Point Observatory", timezone="US/Mountain")
observation_constraints = [
    ap.AltitudeConstraint(30 * u.deg, 80 * u.deg),
    ap.AirmassConstraint(2),
    # ap.AtNightConstraint.twilight_civil(),
]
session_beg, session_end = observer.tonight(Time("2024-09-16", format="iso", scale="utc"), horizon=-6 * u.deg)
session_end -= (session_end - session_beg) / 2 # only the A half of night

standin_parallax = 0.003 * u.arcsec
gaia_epoch_offset = Time.now() - Time("2016-01-01")

wds["Used"] = False
targets = []
num_close, num_far = 0, 0
hour_angle = u.Quantity(0, unit=u.deg)
while num_close < num_targets or num_far < num_targets:
    binaries = wds[
        binary_criteria
        & (wds["RA"] >= hour_angle)
        & (wds["RA"] < hour_angle + 15 * u.deg)
        & (wds["Separation"] > 0)
        & (wds["Used"] == False)]
    binaries.sort("Mag Avg")
    print(f"RA={Angle(hour_angle).hour:.0f}h has {len(binaries)} binaries, found {num_close} close and {num_far} far so far")
    for binary in binaries:
        print(f"Looking at {binary["ID"]}, Mag {binary["Mag A"]:.1f}", end=" ")
        wds["Used"][np.where(wds["ID"] == binary["ID"])] = True
        wds_coord = SkyCoord(binary["RA"], binary["Dec"], unit="deg")
        observable = ap.is_observable(observation_constraints, observer, wds_coord, (session_beg, session_end))
        if not observable:
            print("Not observable")
            continue
        # wds_gaia_epoch = wds_coord.apply_space_motion(dt=gaia_epoch_offset)
        binary_query = (
            f"select *"#DESIGNATION, ra, dec, parallax, astrometric_n_good_obs_al, ruwe, phot_g_mean_mag "
            f"  from gaiadr3.gaia_source "
            f"  WHERE CONTAINS( "
            f"      POINT('ICRS', ra, dec), "
            f"      CIRCLE('ICRS', {wds_coord.ra.value}, {wds_coord.dec.value}, {binary_match_cone.to(u.deg).value})) = 1 "
            f"    and phot_g_mean_mag < {binary["Mag A"] + binary_match_mag} "
            f"    and phot_g_mean_mag > {binary["Mag A"] - binary_match_mag} "
        )
        binary_matches = Gaia.launch_job(binary_query).get_results()
        match len(binary_matches):
            case 0:
                print("Not found")
                continue
            case 1:
                pass
            case _:
                print(f"Too many Gaia stars ({len(binary_matches)})")
                # print(binary["Mag A"], binary["Mag B"])
                # print(binary_matches[["ra", "dec", "phot_g_mean_mag"]])
                continue
        gaia_match = binary_matches[0]
        gaia_mag = gaia_match["phot_g_mean_mag"]
        if abs(gaia_mag - binary["Mag A"]) > binary_match_mag:
            print("Mag too different")
            continue
        if not (binary_parallax := gaia_match["parallax"] > 0):
            binary_parallax = standin_parallax
        gaia_coord = SkyCoord(
            gaia_match["ra"],
            gaia_match["dec"],
            binary_parallax,
            pm_ra_cosdec=gaia_match["pmra"] * u.mas / u.yr,
            pm_dec=gaia_match["pmdec"] * u.mas / u.yr,
            unit="deg")

        # we now have the gaia version of the binary
        # look for nearby point sources of similar magnitude
        point_query = (
            f"select * "
            f"  from gaiadr3.gaia_source "
            f"  where CONTAINS( "
            f"      POINT('ICRS', ra, dec), "
            f"      CIRCLE('ICRS', {gaia_coord.ra.value}, {gaia_coord.dec.value}, {point_match_sep_max.to(u.deg).value})) = 1 "
            f"    and phot_g_mean_mag > {gaia_mag - point_match_mag} "
            f"    and phot_g_mean_mag < {gaia_mag + point_match_mag} "
        )
        point_matches = Gaia.launch_job(point_query).get_results()
        if len(point_matches) == 0:
            print("No point sources")
            continue
        target_found = False
        for point in point_matches:
            if not (point_parallax := point["parallax"] > 0):
                point_parallax = standin_parallax
            point_coord = SkyCoord(
                point["ra"],
                point["dec"],
                point_parallax,
                pm_ra_cosdec=point["pmra"] * u.mas / u.yr,
                pm_dec=point["pmdec"] * u.mas / u.yr,
                unit="deg")
            if point_coord.separation(gaia_coord) < 1 * u.mas:
                print(f"Skipping gaia match of binary (of {len(point_matches)} total sources)")
                continue
            point_sep = gaia_coord.separation(point_coord)
            if point_sep < point_match_sep_min:
                print(f"Skipping point source {point_sep} away")
                continue
            target = {}
            target["Binary WDS ID"] = binary["ID"]
            binary_coord_now = gaia_coord.apply_space_motion(dt=gaia_epoch_offset)
            point_coord_now = point_coord.apply_space_motion(dt=gaia_epoch_offset)
            target["Binary RA"] = binary_coord_now.ra.to_string(unit=u.hour, precision=2)
            target["Binary Dec"] = binary_coord_now.dec.to_string(unit=u.deg, precision=2)
            target["Binary Sep"] = binary["Separation"].to(u.arcsec)
            target["Binary PA"] = binary["Position Angle"]
            target["Binary Mag A"] = binary["Mag A"]
            target["Binary Mag B"] = binary["Mag B"]
            target["Point ID"] = point["DESIGNATION"]
            target["Point RA"] = point_coord_now.ra.to_string(unit=u.hour, precision=2)
            target["Point Dec"] = point_coord_now.dec.to_string(unit=u.deg, precision=2)
            target["Point Sep"] = f"{point_sep.dms[1]:.0f}m{point_sep.dms[2]:02.0f}s"
            target["Point PA"] = gaia_coord.position_angle(point_coord).to(u.deg)
            target["Point Mag"] = point["phot_g_mean_mag"]
            targets.append(target)
            if point_match_sep_min < point_sep <= point_match_sep_mid:
                num_close += 1
            elif point_match_sep_mid < point_sep <= point_match_sep_max:
                num_far += 1
            else:
                print("Shouldn't get here")
                print(point)
                print(point_sep).to(u.arcsec)
            print(f"Match found: {point["DESIGNATION"]}, target count={len(targets)}")
            target_found = True
        # if target_found:
        #     break
    hour_angle = u.Quantity((hour_angle.value + 15) % 360, unit=u.deg)
wds.remove_column("Used")

diffit_targets = QTable(targets)
diffit_targets["Binary Sep"].info.format = ".2f"
diffit_targets["Binary PA"].info.format = ".2f"
diffit_targets["Point PA"].info.format = ".0f"
diffit_targets["Point Mag"].info.format = ".2f"
diffit_targets.sort("Point Sep")
diffit_targets.write("DIFFIT Targets.ecsv", overwrite=True)
diffit_targets.write("DIFFIT Targets.csv", overwrite=True)
diffit_targets

RA=0h has 42 binaries, found 0 close and 0 far so far
Looking at 00318+5431, Mag 5.3 Skipping gaia match of binary (of 1 total sources)
Looking at 00315-6257, Mag 5.8 Not observable
Looking at 00568+6022, Mag 6.1 Skipping gaia match of binary (of 1 total sources)
Looking at 00243+5201, Mag 6.0 Skipping gaia match of binary (of 1 total sources)
Looking at 00373-2446, Mag 6.6 Not found
Looking at 00003-4417, Mag 6.8 Not observable
Looking at 00284-2020, Mag 7.2 Skipping gaia match of binary (of 1 total sources)
Looking at 00153+4412, Mag 7.3 Skipping gaia match of binary (of 1 total sources)
Looking at 00327+3851, Mag 7.7 Skipping gaia match of binary (of 1 total sources)
Looking at 00290+4825, Mag 7.7 Not found
Looking at 00452+0015, Mag 7.8 Skipping gaia match of binary (of 1 total sources)
Looking at 00374-3717, Mag 7.7 Not observable
Looking at 00358+4901, Mag 7.5 Skipping gaia match of binary (of 1 total sources)
Looking at 00345-0433, Mag 7.6 Skipping gaia match of binary (of 1 tot

Binary WDS ID,Binary RA,Binary Dec,Binary Sep,Binary PA,Binary Mag A,Binary Mag B,Point ID,Point RA,Point Dec,Point Sep,Point PA,Point Mag
,,,arcsec,,,,,,,,deg,
str10,str12,str13,float64,int32,float64,float64,str28,str12,str13,str5,float64,float32
01198-0031,1h19m48.35s,-0d30m31.32s,0.10,16.00,7.4,7.6,Gaia DR3 2533762977853808640,1h19m48.31s,-0d30m32.78s,0m02s,201,6.13
18455+0530,18h45m28.54s,5d29m59.16s,0.10,323.00,7.48,7.48,Gaia DR3 4285283042432285568,18h45m28.38s,5d30m00.48s,0m03s,300,6.42
00174+0853,0h17m24.46s,8d52m35.04s,0.10,136.00,8.38,7.78,Gaia DR3 2752315446728243328,0h17m24.68s,8d52m37.29s,0m04s,54,7.01
19392-1654,19h39m13.02s,-16d54m27.74s,0.10,-1.00,7.9,7.9,Gaia DR3 4180103760088470784,19h39m12.45s,-16d54m33.31s,0m10s,236,7.39
19490+4423,19h49m00.91s,44d22m45.17s,0.20,34.00,8.2,8.7,Gaia DR3 2078942571333388544,19h49m01.22s,44d22m36.11s,0m10s,159,8.28
16205-2007,16h20m30.55s,-20d06m52.65s,0.15,171.00,8.1,8.5,Gaia DR3 6244711994020423424,16h20m30.24s,-20d07m04.54s,0m13s,200,7.44
18002+8000,18h00m09.63s,80d00m17.72s,0.10,316.00,6.22,6.69,Gaia DR3 2294405721759384064,18h00m03.99s,80d00m06.31s,0m19s,232,5.93
01057+2128,1h05m41.06s,21d28m22.91s,0.10,137.00,5.7,6.6,Gaia DR3 2790497465471261568,1h05m41.81s,21d27m55.22s,0m30s,159,5.55


In [72]:
target_groups = [
    ("DIFFIT Targets 0 to 4min.tui", (1 * u.arcsec, 4 * u.arcmin), "blue"),
    ("DIFFIT Targets 4 to 8min.tui", (4 * u.arcmin, 8 * u.arcmin), "red"),
]

diffit_targets["Point Sep Deg"] = [ Angle("0d" + sep) for sep in diffit_targets["Point Sep"]]
for filename, (lo, hi), color in target_groups:
    with open(filename, "w") as f:
        f.write(f"CSys=FK5; Date=2000; RotType=Object; RotAng=0; RotWrap=Nearest; Keep=Arc; dispcolor={color}\n")
        subset = diffit_targets[(lo < diffit_targets["Point Sep Deg"]) & (diffit_targets["Point Sep Deg"] <= hi)]
        for target in subset:
            ra_colons = target["Binary RA"].replace("h", ":").replace("m", ":").replace("s", "")
            dec_colons = target["Binary Dec"].replace("d", ":").replace("m", ":").replace("s", "")
            f.write(f'"{target["Binary WDS ID"]:<10s}"       {ra_colons:<14s} {dec_colons:<18s} PM=0.00000, 0.00000; Magnitude={target["Binary Mag A"]:5.2f}\n')
        print(f"Wrote {len(subset)} targets to {filename}")


Wrote 30 targets to DIFFIT Targets 0 to 4min.tui
Wrote 52 targets to DIFFIT Targets 4 to 8min.tui
